In [1]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import json
import pickle

import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

import random

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jbjb2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


In [2]:
#what is lemmatization: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
lemmatizer = WordNetLemmatizer() 

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
#preprocessing data

#tokenizing data: breaking whole text into small parts
#iterating through patterns and tokenizing sentences
#classes list collects tags

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        
        #add to out classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
#lemmatize, lower each word, and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
#sort classes
classes = sorted(list(set(classes)))
#documents=combination between patterns and intents
print(len(documents), 'documents')
#classes=intents
print(len(classes), 'classes', classes)
#words=all words, vocab
print(len(words), 'unique lemmatized words', words)

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

40 documents
6 classes ['answer', 'goodbye', 'greeting', 'options', 'predicting', 'thanks']
70 unique lemmatized words ["'s", ',', 'a', 'actually', 'answer', 'anyone', 'are', 'awesome', 'be', 'broken', 'bye', 'can', 'chatting', 'correct', 'could', 'day', 'do', 'doe', 'enter', 'for', 'give', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'image', 'incorrect', 'insert', 'inserting', 'instruction', 'is', 'later', 'link', 'me', 'my', "n't", 'next', 'nice', 'not', 'offered', 'on', 'option', 'other', 'picture', 'provide', 'see', 'send', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'what', 'with', 'work', 'working', 'wrong', 'you']


In [5]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        # output is a '0' for each tag and '1' for current tag (for each pattern)
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer
#contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/50
2800/2800 [==============================] - 1s 249us/step - loss: 0.4597 - accuracy: 0.8589
Epoch 2/50
2800/2800 [==============================] - 1s 216us/step - loss: 0.0350 - accuracy: 0.9932
Epoch 3/50
2800/2800 [==============================] - 1s 217us/step - loss: 0.0172 - accuracy: 0.9964
Epoch 4/50
2800/2800 [==============================] - 0s 174us/step - loss: 0.0114 - accuracy: 0.9971
Epoch 5/50
2800/2800 [==============================] - 1s 180us/step - loss: 0.0103 - accuracy: 0.9986
Epoch 6/50
2800/2800 [==============================] - 1s 255us/step - loss: 0.0116 - accuracy: 0.9975
Epoch 7/50
2800/2800 [==============================] - 1s 281us/step - loss: 0.0065 - accuracy: 0.9993
Epoch 8/50
2800/2800 [==============================] - 0s 174us/step - loss: 0.0051 - accuracy: 0.9993
Epoch 9/50
2800/2800 [==============================] - 1s 243us/step - loss: 0.0045 - accuracy: 0.9993
Epoch 10/50
2800/2800 [==============================] - 0s 167u